In [1]:
import os
import sys
project_root_dir = os.path.relpath(os.path.join('..', '..'), os.curdir)
if project_root_dir not in sys.path:
    sys.path += [project_root_dir]
from src.data.dataload import *
from src.models.bcn_model import *
from pprint import pprint

# Data<br>
Loading data

In [2]:
data = load_sst()
print(f'loading data {data.NAME} (sentence column: {data.SENTENCE}, target column: {data.TARGET})')
train, val, test = data.train_val_test
train

loading data sst (sentence column: sentence, target column: label)


,sentence,label
0,The Rock is destined to be the 21st Century 's...,3
1,The gorgeously elaborate continuation of `` Th...,4
2,Singer/composer Bryan Adams contributes a slew...,3
3,You 'd think by now America would have had eno...,2
4,Yet the act is still charming here .,3
...,...,...
8539,A real snooze .,0
8540,No surprises .,1
8541,We 've seen the hippie-turned-yuppie plot befo...,3
8542,Her fans walked out muttering words like `` ho...,0


# Model<br>
Fine-tuning and loading BCN

In [3]:
bcn = BCNModel()
bcn.load_model(data)
bcn.model

BiattentiveClassificationNetwork(
  (_text_field_embedder): BasicTextFieldEmbedder(
    (token_embedder_tokens): Embedding()
  )
  (_embedding_dropout): Dropout(p=0.25, inplace=False)
  (_pre_encode_feedforward): FeedForward(
    (_activations): ModuleList(
      (0): ReLU()
    )
    (_linear_layers): ModuleList(
      (0): Linear(in_features=300, out_features=300, bias=True)
    )
    (_dropout): ModuleList(
      (0): Dropout(p=0.25, inplace=False)
    )
  )
  (_encoder): LstmSeq2SeqEncoder(
    (_module): LSTM(300, 300, batch_first=True, bidirectional=True)
  )
  (_integrator): LstmSeq2SeqEncoder(
    (_module): LSTM(1800, 300, batch_first=True, bidirectional=True)
  )
  (_integrator_dropout): Dropout(p=0.1, inplace=False)
  (_self_attentive_pooling_projection): Linear(in_features=600, out_features=1, bias=True)
  (_output_layer): Maxout(
    (_linear_layers): ModuleList(
      (0): Linear(in_features=2400, out_features=4800, bias=True)
      (1): Linear(in_features=1200, out_featu

In [4]:
pprint(bcn.finetune_report())

{'best_epoch': 4,
 'best_validation_accuracy': 0.43869209809264303,
 'best_validation_accuracy3': 0.9146230699364214,
 'best_validation_loss': 1.224069242676099,
 'epoch': 4,
 'peak_gpu_0_memory_MB': 741.2734375,
 'peak_worker_0_memory_MB': 2449.7265625,
 'training_accuracy': 0.47495318352059923,
 'training_accuracy3': 0.9188904494382022,
 'training_duration': '0:00:51.548481',
 'training_epochs': 4,
 'training_gpu_0_memory_MB': 741.2734375,
 'training_loss': 1.1899946727031885,
 'training_start_epoch': 0,
 'training_worker_0_memory_MB': 2449.7265625,
 'validation_accuracy': 0.43869209809264303,
 'validation_accuracy3': 0.9146230699364214,
 'validation_loss': 1.224069242676099}


Prediction

In [5]:
print(f'Individual prediction for {data.NAME}')
bcn.predict(test.sentence[0])
print(bcn.predict(test[data.SENTENCE][0]))

Individual prediction for sst
     logits  class_probabilities label
0  0.620440             0.016874     1
1  4.414236             0.749643     1
2  2.922851             0.168715     1
3 -3.867865             0.000190     1
4  1.962503             0.064577     1


In [6]:
print(f'Batch prediction for {data.NAME}')
bcn.predict_batch_df(test[:100], input_col=data.SENTENCE)
print(bcn.predict_batch(test[data.SENTENCE][:100]))

Batch prediction for sst
                                               logits  \
0   [0.6204397678375244, 4.414236068725586, 2.9228...   
1   [1.6763371229171753, -0.020200669765472412, 0....   
2   [1.2694412469863892, 1.169205665588379, 0.8316...   
3   [1.9059851169586182, -0.21952968835830688, 0.4...   
4   [3.5735950469970703, -2.2221274375915527, 0.29...   
..                                                ...   
95  [1.1532161235809326, -0.2188836634159088, 0.24...   
96  [1.259469747543335, 1.6719794273376465, 1.1938...   
97  [3.3070015907287598, -1.3257590532302856, 0.48...   
98  [0.9012832641601562, 1.8801426887512207, 1.477...   
99  [3.3964028358459473, -0.5213455557823181, 0.84...   

                                  class_probabilities label  
0   [0.016874458640813828, 0.7496432662010193, 0.1...     1  
1   [0.5133780837059021, 0.09411099553108215, 0.15...     3  
2   [0.3514782786369324, 0.31795579195022583, 0.22...     3  
3   [0.5327665209770203, 0.063597142696380

In [7]:
print(f'Individual label prediction for {data.NAME}')
print(bcn.predict_label(test.sentence[0]))
print(f'Batched label prediction for {data.NAME}')
bcn.predict_label_batch_df(test[:100], input_col=data.SENTENCE)
print(bcn.predict_label_batch(test.sentence[:100]))

Individual label prediction for sst
[1]
Batched label prediction for sst
[1 3 3 3 4 1 3 1 3 4 3 3 4 1 3 1 4 3 4 4 4 3 3 3 3 3 3 3 1 0 4 1 1 3 1 3 3
 3 1 1 3 1 3 4 3 1 4 3 3 3 3 1 3 3 3 1 3 1 4 3 3 3 3 3 3 1 3 4 4 3 1 3 4 1
 3 1 3 4 3 3 3 3 3 4 3 3 3 3 3 1 3 3 1 3 1 3 1 4 1 3]


In [8]:
print(f'Individual proba prediction for {data.NAME}')
print(bcn.predict_proba(test.sentence[0]))
print(f'Batched label prediction for {data.NAME}')
bcn.predict_proba_batch_df(test[:10], input_col=data.SENTENCE)
print(bcn.predict_proba_batch(test.sentence[:10]))

Individual proba prediction for sst
0    0.016874
1    0.749643
2    0.168715
3    0.000190
4    0.064577
Name: class_probabilities, dtype: float64
Batched label prediction for sst
[[1.68744586e-02 7.49643266e-01 1.68715343e-01 1.89663609e-04
  6.45773411e-02]
 [5.13378143e-01 9.41109732e-02 1.54911533e-01 2.15201989e-01
  2.23974362e-02]
 [3.51478279e-01 3.17955792e-01 2.26855606e-01 4.66247834e-02
  5.70855439e-02]
 [5.32766581e-01 6.35971352e-02 1.22242726e-01 2.67004907e-01
  1.43886181e-02]
 [1.73870429e-01 5.28658624e-04 6.58263918e-03 8.18903685e-01
  1.14566858e-04]
 [1.15683466e-01 4.57678586e-01 1.81993723e-01 1.85341705e-02
  2.26110026e-01]
 [5.72937071e-01 1.63806468e-01 2.01391816e-01 5.00272103e-02
  1.18374769e-02]
 [1.08628422e-01 5.18325627e-01 3.26272458e-01 3.83080798e-03
  4.29427028e-02]
 [5.12488544e-01 1.81336177e-03 1.73128508e-02 4.68215287e-01
  1.69911014e-04]
 [3.72175813e-01 1.37320673e-02 5.14379516e-02 5.58647037e-01
  4.00706055e-03]]


In [9]:
print(f'Individual logits prediction for {data.NAME}')
print(bcn.predict_logits(test.sentence[0]))
print(f'Batched logits prediction for {data.NAME}')
bcn.predict_logits_batch_df(test[:10], input_col=data.SENTENCE)
print(bcn.predict_logits_batch(test.sentence[:10]))

Individual logits prediction for sst
0    0.620440
1    4.414236
2    2.922851
3   -3.867865
4    1.962503
Name: logits, dtype: float64
Batched logits prediction for sst
[[ 0.62043977  4.41423607  2.92285156 -3.86786509  1.96250224]
 [ 1.67633724 -0.02020071  0.47817868  0.80690157 -1.45572901]
 [ 1.26944125  1.16920567  0.83160698 -0.75057447 -0.54815578]
 [ 1.90598536 -0.21952975  0.43391067  1.21516895 -1.70566058]
 [ 3.57359529 -2.22212744  0.29972091  5.12325144 -3.75131178]
 [ 0.29771441  1.67302382  0.75082886 -1.53352714  0.96787834]
 [ 2.24179268  0.98970252  1.19626915 -0.1964162  -1.63771272]
 [ 1.15510476  2.71777534  2.25490451 -2.18975234  0.22703829]
 [ 4.08390713 -1.56018877  0.6960777   3.99355698 -3.92785168]
 [ 2.22267056 -1.07696199  0.24368057  2.62882209 -2.3086381 ]]
